In [46]:
# Recources

# https://www.tensorflow.org/hub
# https://keras.io/api/applications/
# https://www.kaggle.com/models/tensorflow/resnet-50

In [47]:
import numpy as np
import cv2
import os

import matplotlib as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers # type: ignore
from tensorflow.keras.models import Sequential # type: ignore


In [48]:
dataset_path = 'C:\\Users\\Karim\\OneDrive\\Documents\\UNI\\SPRING 24\\CMPS 261\\Project\\Food-Item-Recognition\\data'

# Create an ImageDataGenerator for preprocessing images
data_generator = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0,1]
    validation_split=0.2  # Split data into training and validation sets
)

# Load and preprocess training data
train_data_flow = data_generator.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Use training subset
)

# Load and preprocess validation data
val_data_flow = data_generator.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use validation subset
)

Found 4107 images belonging to 9 classes.
Found 1024 images belonging to 9 classes.


In [49]:
# Read the category file and parse category names and IDs
category_file_path = os.path.join(dataset_path, 'category.txt')
category_mapping = {}
with open(category_file_path, 'r') as f:
    next(f)  # Skip the header line
    for line in f:
        category_id, category_name = line.strip().split('\t')
        category_mapping[int(category_id)] = category_name

In [56]:
# Load the model from TensorFlow Hub
model_url = "https://www.kaggle.com/models/tensorflow/resnet-50/TensorFlow2/classification/1"
hub_layer = hub.KerasLayer(model_url, input_shape=(224, 224, 3))

# Create a Sequential model
model = tf.keras.Sequential()

# Add the TensorFlow Hub layer to the Sequential model
model.add(hub_layer)

# Build the Sequential model
model.build((None, 224, 224, 3))

# Summary of the model
model.summary()


ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x00000190C6B8AD20> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [ ]:

# Define additional layers for classification
model = keras.Sequential([
    resnet,
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(9, activation='softmax')  # Assuming 9 classes
])

In [ ]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Train the model
history = model.fit(train_data_flow, validation_data=val_data_flow, epochs=10)

In [ ]:
# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()